# Segmented Trajectory Subsetter documentation and examples:

### Contact:

**Slack:** #sds, #harmony-service-providers

**Email:** david.p.auty@nasa.gov, owen.m.littlejohns@nasa.gov.

The scope of this notebook is to demonstrate the capabilities the Segmented Trajectory Subsetter.

### Capabilities:

* Variable subsetting.
* Temporal subsetting.
* Bounding box spatial subsetting.
* Polygon spatial subsetting.

### Distinctions from other services:

* Works on trajectory, rather than swath data (1-D, usually with a time dimension).
* Preserves continuity of photon segment indices.
* Configured for Cloud-hosted GEDI L4A test collection.

### Segmented trajectories:

![](images/segmented_trajectory.png)

**Figure 1:** Annotated image from Earthdata Search to illustrate segmented trajectory collections. The granule selected has a trajectory split into continuous segments. In the example above, the bounding box shows a spatial subset request that requires the removal of some of the segments. The output also will not be continuous and so the segments will need to be re-indexed to ensure the output contains continuous indices. (Note: Real trajectory segments are much shorter than those shown in this image)

### Heritage:

The Harmony service is a new, Python-based wrapper on the existing SDPS Trajectory Subsetter on-premises service. This 
on-premises service is underpinned by a C++ binary that is invoked via the EOSDIS Service Interface (ESI) and specific tool adapters. The Trajectory Subsetter has been used on-premises to support trajectory collections for missions such as SMAP and ICESat-2.

### Architecture:

Harmony, Python-based infrastructure replaces the ESI and tool adapters in the on-premises stack:

![](images/TS_sequence_diagram.png)
**Figure 2:** Sequence flow of the Trajectory Subsetter Harmony backend service. A Harmony request is received for a collection associated with that service. Harmony constructs a message and sends it to the Trajectory Subsetter. The `HarmonyAdapter` class within the backend service downloads the granule and then translates the Harmony message parameters into arguments for a call to the C++ binary. The C++ binary is invoked, with its output status captures, to ensure success. Finally, the transformed output is staged to S3.


### Requirements

* Trajectory data.
* Cloud-hosted data (best ingested via Cumulus).
* UMM-C and UMM-G records, again best created via Cumulus, including `GET DATA` related URL.
* UMM-Var records associated with the collection.
* A UMM-S record (for each production DAAC provider), with that UMM-S concept ID listed against the Trajectory Subsetter service in [services.yml](https://github.com/nasa/harmony/blob/main/config/services.yml).
* Association between the UMM-C record(s) for your collection(s) and the UMM-S record.

### Examples

The following examples will use `harmony-py` (see: [here](https://github.com/nasa/harmony-py)) to make requests to Harmony. `harmony-py` is distributed via the Python Package Index ([PyPI](https://pypi.org/project/harmony-py/)), and can be installed to your current Python environment via Pip:

```bash
$ pip install harmony-py
```

Other Python packages used by this notebook (for evaluating output):

* cartopy
* matplotlib
* netCDF4

#### Initial set-up

The following cell imports standard classes from `harmony-py` and establishes a client to make requests to the UAT environment of Harmony. It also sets up a collection instance for the GEDI L4A collection, as hosted in the EEDTEST provider (C1242267295-EEDTEST).

### Set up imports and variables

In [ ]:
from datetime import datetime
from os import replace
from os.path import exists
from shutil import move

from harmony import BBox, Client, Collection, Environment, Request
from netCDF4 import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt


gedi_l4a = Collection(id='C1242267295-EEDTEST')
file_root_name = 'GEDI_L4A'
variables = ['/BEAM0000/agbd']
granule_id = 'G1242274838-EEDTEST'

harmony_client = Client(env=Environment.UAT)

### Helper functions:

In [ ]:
def print_all_variables(dataset):
    """ A helper function to traverse all groups in a file and
        print the variables present, along with their sizes.

    """
    print('\nVariables present:\n')

    for group in dataset.groups.values():
        for variable in group.variables.values():
            print(f'{group.path}/{variable.name}, {variable.shape}')

    print('\n')


def get_geo_plot_fig_and_ax(extents):
    """ A helper function to retrieve the figure and axes for a plot
        showing landmasses, oceans and country borders.

    """
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent(extents, crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.RIVERS)
    return fig, ax

### Variable subsetting:

The GEDI L4A collection has granules that are at least 1 GB in size. The Trajectory Subsetter provides the capability to perform a variable subset, to reduce the size of the output product. The variable subset will include the requested variable:

* `/BEAM0000/agbd`, above ground biomass density.

As well as the added support variables:

* `/BEAM0000/delta_time`, the temporal coordinate for the science variable.
* `/BEAM0000/lat_lowestmode`, the latitude coordinate for the science variable.
* `/BEAM0000/lon_lowestmode`, the longitude coordinate for the science variable.

And configured ancillary variable:

* `/BEAM0000/shot_number`, the shot number.

In [ ]:
variable_subset_request = Request(collection=gedi_l4a, variables=variables, granule_id=[granule_id])
variable_subset_job_id = harmony_client.submit(variable_subset_request)
variable_local_file_name = f'{file_root_name}_variable.h5'

print(f'Processing job: {variable_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(variable_subset_job_id, overwrite=True)]:
    if filename.endswith('.h5'):
        print(f'Downloaded: {filename}')
        h5_filename = filename

replace(h5_filename, variable_local_file_name)
print(f'Saved output to: {variable_local_file_name}')

### Checking the output:

In [ ]:
fig, ax = get_geo_plot_fig_and_ax([-180, 180, -90, 90])

with Dataset(variable_local_file_name, 'r') as variable_dataset:
    print_all_variables(variable_dataset)
    variable_latitude = variable_dataset['/BEAM0000/lat_lowestmode'][:]
    variable_longitude = variable_dataset['/BEAM0000/lon_lowestmode'][:]

plt.plot(variable_longitude, variable_latitude, color='red', linewidth=2, transform=ccrs.Geodetic())

### Temporal subsetting:

The Trajectory Subsetter accepts a single temporal range. In the request below, the temporal subset has been combined with a variable subset to maintain a small output product.

The temporal range specified is from 2020-07-25T01:00:00 to 2020-07-25T02:00:00.

The full temporal range of the granule is 2020-07-25T01:22:56 to 2020-07-25T02:55:47, so the expected output from the temporal subset should be approximately the first third of the data.

In [ ]:
temporal_range = {'start': datetime(2020, 7, 25, 1, 0, 0), 'stop': datetime(2020, 7, 25, 2, 0, 0)}

temporal_subset_request = Request(collection=gedi_l4a, temporal=temporal_range,
                                  variables=variables, granule_id=[granule_id])
temporal_subset_job_id = harmony_client.submit(temporal_subset_request)
temporal_local_file_name = f'{file_root_name}_temporal.h5'

print(f'Processing job: {temporal_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(temporal_subset_job_id, overwrite=True)]:
    if filename.endswith('.h5'):
        print(f'Downloaded: {filename}')
        h5_filename = filename

replace(h5_filename, temporal_local_file_name)
print(f'Saved output to: {temporal_local_file_name}')

### Checking the output:

In [ ]:
fig, ax = get_geo_plot_fig_and_ax([-180, 180, -90, 90])

with Dataset(temporal_local_file_name, 'r') as temporal_dataset:
    print_all_variables(temporal_dataset)
    temporal_latitude = temporal_dataset['/BEAM0000/lat_lowestmode'][:]
    temporal_longitude = temporal_dataset['/BEAM0000/lon_lowestmode'][:]

plt.plot(temporal_longitude, temporal_latitude, color='red', linewidth=2, transform=ccrs.Geodetic())

### Bounding box spatial subsetting:

The Trajectory Subsetter accepts a single bounding box. The request below combines a bounding box with a variable subset to maintain a small output product.

The bounding box specified is:

* 45 ≤ latitude (degrees north) ≤ 55
* -120 ≤ longitude (degrees east) ≤ -100

Things to check in output:

* The four requested variables are present in the output.
* The variables all have the same size, which should be smaller than the variable subset only.
* The `/BEAM0000/lat_lowestmode` only has values in the range of the bounding box.
* The `/BEAM0000/lon_lowestmode` only has values in the range of the bounding box.

In [ ]:
bounding_box = BBox(w=-120, s=45, e=-100, n=55)

bbox_subset_request = Request(collection=gedi_l4a, spatial=bounding_box,
                              granule_id=[granule_id], variables=variables)
bbox_subset_job_id = harmony_client.submit(bbox_subset_request)
bbox_local_file_name = f'{file_root_name}_bbox.h5'

print(f'Processing job: {bbox_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(bbox_subset_job_id, overwrite=True)]:
    if filename.endswith('.h5'):
        print(f'Downloaded: {filename}')
        h5_filename = filename

replace(h5_filename, bbox_local_file_name)
print(f'Saved output to: {bbox_local_file_name}')

### Checking the results

In [ ]:
fig, ax = get_geo_plot_fig_and_ax([-130, -90, 35, 65])

with Dataset(bbox_local_file_name, 'r') as bbox_dataset:
    print_all_variables(bbox_dataset)
    bbox_latitude = bbox_dataset['/BEAM0000/lat_lowestmode'][:]
    bbox_longitude = bbox_dataset['/BEAM0000/lon_lowestmode'][:]

bbox_longitudes = [bounding_box.w, bounding_box.w, bounding_box.e, bounding_box.e, bounding_box.w]
bbox_latitudes = [bounding_box.s, bounding_box.n, bounding_box.n, bounding_box.s, bounding_box.s]

plt.plot(bbox_longitudes, bbox_latitudes, color='green', linewidth=2, transform=ccrs.Geodetic())
plt.plot(bbox_longitude, bbox_latitude, color='red', linewidth=2, transform=ccrs.Geodetic())

### Polygon spatial subsetting:

The Trajectory Subsetter accepts a GeoJSON shape file, instructing it to perform polygon spatial subsetting. The request below combines a polygon of Egypt with a variable subset to maintain a small output product.

Things to check in the output:

* The four requested variables are present in the output.
* The variables all have the same size, which should be smaller than the variable subset only.
* The `/BEAM0000/lat_lowestmode` only has values in the range of the polygon.
* The `/BEAM0000/lon_lowestmode` only has values in the range of the polygon.

In [ ]:
egypt_granule_id = 'G1242274852-EEDTEST'

polygon_subset_request = Request(collection=gedi_l4a, granule_id=[egypt_granule_id],
                                 variables=variables, shape='EGY.geo.json')
polygon_subset_job_id = harmony_client.submit(polygon_subset_request)
polygon_local_file_name = f'{file_root_name}_polygon.h5'


print(f'Processing job: {polygon_subset_job_id}')

for filename in [file_future.result()
                 for file_future
                 in harmony_client.download_all(polygon_subset_job_id, overwrite=True)]:
    if filename.endswith('.h5'):
        print(f'Downloaded: {filename}')
        h5_filename = filename

replace(h5_filename, polygon_local_file_name)
print(f'Saved output to: {polygon_local_file_name}')

### Checking the output:

In [ ]:
fig, ax = get_geo_plot_fig_and_ax([20, 40, 19, 35])

with Dataset(polygon_local_file_name, 'r') as polygon_dataset:
    print_all_variables(polygon_dataset)
    polygon_latitude = polygon_dataset['/BEAM0000/lat_lowestmode'][:]
    polygon_longitude = polygon_dataset['/BEAM0000/lon_lowestmode'][:]

plt.plot(polygon_longitude, polygon_latitude, color='red', linewidth=2, transform=ccrs.Geodetic())